Task 1 - Implemenet NN Layers
-----

### 1.1 Fully Connected Layer
Before we get started, let's recall what happens in the forward pass of a full-connected layer.

In [1]:
import math
import numpy as np

class Linear():
    """A fully-connected NN layer.
    Parameters:
    -----------
    n_units: int
        The number of neurons in the layer.
    input_shape: tuple
        The expected input shape of the layer. For dense layers a single digit specifying
        the number of features of the input. Must be specified if it is the first layer in
        the network.
    """
    def __init__(self, n_units, input_shape=None):
        # For simplisity, we omit optimizer in our homework.
        # Therefore, you do not need to worry about parameter update.
        self.layer_input = None
        self.input_shape = input_shape
        self.n_units = n_units
        self.trainable = True
        self.W = None
        self.b = None
        self.initialize()

    def initialize(self):
        # Initialize the weights
        limit = 1 / math.sqrt(self.input_shape[0])
        self.W  = np.random.uniform(-limit, limit, (self.input_shape[1], self.n_units))
        self.b = np.zeros((1, self.n_units))

    def forward_pass(self, inp):
        self.layer_input = inp
        return np.dot(inp, self.W) + self.b

Below we provided some helper functions that might be useful:

In [2]:
def SE(out, target):
    '''
    return square error.
    '''
    return 0.5 * (target - out)**2

def get_target(inp, W, b):
    '''
    W and b are assumed ideal weights and bias.
    '''
    return np.dot(inp, W) + b

def grad_check(layer, inp, W, b):
    '''
    calculate gradient from numerical method, we compare the analytical gradient and numerical gradient.
    
    We say your calculated gradients are correct when the mean square error between 
    standard gradient and your gradient is below some threshold.
    
    return true when gradients of W, b and inp are calculated correctly.
    '''
    res = True
    target = get_target(inp, W, b)
    out = layer.forward_pass(inp)
    y = SE(target, out)
    loss = target - out
    accum_grad = layer.backward_pass(loss)
    
    W_shape = layer.W.shape
    b_shape = layer.b.shape
    inp_shape = inp.shape
    
    limit = 1e-6
    threshold = 1e-8 * inp_shape[0]**2
    
    W_diff = np.zeros(W_shape)
    for i in range(W_shape[0]):
        noise = np.random.rand(W_shape[1]) * limit
        layer.W[i,:] += noise
        out2 = layer.forward_pass(inp)
        y2 = SE(target, out2)
        W_diff[i,:] = np.sum(y - y2, axis=0) / noise
        layer.W[i,:] -= noise
        
    res &= (np.sum((W_diff - layer.grad_W)**2) < threshold)
    
    noise = np.random.rand(*b_shape) * limit
    layer.b += noise
    out2 = layer.forward_pass(inp)
    y2 = SE(target, out2)
    b_diff = np.sum(y - y2, axis=0) / noise
    layer.b -= noise   
    
    res &= (np.sum((b_diff - layer.grad_b)**2) < threshold)
    
    inp_diff = np.zeros(inp_shape)
    for j in range(inp_shape[1]):
        noise = np.random.rand(inp_shape[0]) * limit
        inp[:,j] += noise
        out2 = layer.forward_pass(inp)
        y2 = SE(target, out2)
        inp_diff[:,j] = np.sum(y - y2, axis=1) / noise
        inp[:,j] -= noise
 
    res &= (np.sum((inp_diff - accum_grad)**2) < threshold) 
    
    return res
    

### Implement the Backward Pass
Now you can start building your own backward function of the fully connected layer.

In [3]:
def backward_pass_fc(self, accum_grad):
    '''
    TODO: Implement the backward_pass_fc here.
    
    Parameter:
    ----------
        accum_grad: gradient propogated back from the next layer
        
    Return:
    ----------
        accum_grad_result: gradient propogated back from the this layer
    '''
    
    self.grad_W = 0
    self.grad_b = 0
    accum_grad_result = np.zeros(self.layer_input.shape)
    
    # the gradient of weights
    self.grad_W = self.layer_input.T.dot(accum_grad)
    
    # the gradient of bias
    grad_out_b = np.ones([1,100])  
    self.grad_b = grad_out_b.dot(accum_grad)
      
    # the gradient of input
    accum_grad_result = accum_grad.dot(self.W.T) 
    
    
    return accum_grad_result

### Test your implementation
Use grad_check to test the correctness of your backward implementation:

In [4]:
Linear.backward_pass = backward_pass_fc

inp = np.random.rand(100,3)
layer = Linear(2, inp.shape)

W = np.random.rand(3,2)
b = np.random.rand(1,2)

if grad_check(layer, inp, W, b):
    print("[INFO] Testing Backward Pass: Pass!")
else:
    print("[WARN] Testing Backward Pass: Fail!")

[INFO] Testing Backward Pass: Pass!


### 1.2 Convolutional Layer
Before we get started, let's recall what happens in the forward pass of a convolutional layer.    

In [5]:
class Conv2D():
    """A 2D Convolution Layer.

    Parameters:
    -----------
    n_filters: int
        The number of filters that will convolve over the input matrix. The number of channels
        of the output shape.
    filter_shape: tuple
        A tuple (filter_height, filter_width).
    input_shape: tuple
        The shape of the expected input of the layer. (batch_size, channels, height, width)
        Only needs to be specified for first layer in the network.
    padding: string
        Either 'same' or 'valid'. 'same' results in padding being added so that the output height and width
        matches the input height and width. For 'valid' no padding is added.
        By default, we use 'same' to test the implementation.
    stride: int
        The stride length of the filters during the convolution over the input.
    """
    
    def __init__(self, n_filters, filter_shape, input_shape, padding='same', stride=1):
        self.n_filters = n_filters
        self.filter_shape = filter_shape
        self.padding = padding
        self.stride = stride
        self.input_shape = input_shape
        self.trainable = True
        self.W = None
        self.w0 = None
        self.initialize()

    def initialize(self):
        # Initialize the weights
        filter_height, filter_width = self.filter_shape
        batch, channels, height, width = self.input_shape
        limit = 1 / math.sqrt(np.prod(self.filter_shape))
        self.W  = np.random.uniform(-limit, limit, size=(self.n_filters, channels, filter_height, filter_width))
        self.w0 = np.zeros((self.n_filters, 1))

    def output_shape(self):
        batch, channels, height, width = self.input_shape
        pad_h, pad_w = determine_padding(self.filter_shape, output_shape=self.padding)
        output_height = (height + np.sum(pad_h) - self.filter_shape[0]) / self.stride + 1
        output_width = (width + np.sum(pad_w) - self.filter_shape[1]) / self.stride + 1
        return self.n_filters, int(output_height), int(output_width)
    
    def forward_pass(self, X):
        batch_size, channels, height, width = X.shape
        self.layer_input = X
        # Turn image shape into column shape
        # (enables dot product between input and weights)
        self.X_col = image_to_column(X, self.filter_shape, stride=self.stride, output_shape=self.padding)
        print("the shape of self.X_col  ", np.shape(self.X_col))
        #the shape of self.X_col   (27, 25)
        # Turn weights into column shape
        self.W_col = self.W.reshape((self.n_filters, -1))
        
        print("the shape of self.W.col   ", np.shape(self.W_col))
        #the shape of self.W.col    (5, 27)
        # Calculate output
        output = self.W_col.dot(self.X_col) + self.w0
        
        print("the shape of output_forward1:  ", np.shape(output))
        # the shape of output_forward1:   (5, 25)
        # Reshape into (n_filters, out_height, out_width, batch_size)
        output = output.reshape(self.output_shape() + (batch_size, ))
        print("the shape of output_forward2:  ", np.shape(output))
        # the shape of output_forward2:   (5, 5, 5, 1)
        # Redistribute axises so that batch size comes first
        print("the shape of output_forward_final:  ", np.shape(output.transpose(3,0,1,2)))
        #
        return output.transpose(3,0,1,2)

Below we provided some helper functions that might be useful:

In [6]:
# Method which turns the image shaped input to column shape.
# Used during the forward pass.
# Reference: CS231n Stanford
def image_to_column(images, filter_shape, stride, output_shape='same'):
    filter_height, filter_width = filter_shape

    pad_h, pad_w = determine_padding(filter_shape, output_shape)
    print("the pad_h:  ", pad_h )
    print("teh pad_w   ", pad_w)
    

    # Add padding to the image
    images_padded = np.pad(images, ((0, 0), (0, 0), pad_h, pad_w), mode='constant')
    
    print("the shape of images_padded  ", np.shape(images_padded))
    
    # Calculate the indices where the dot products are to be applied between weights
    # and the image
    k, i, j = get_im2col_indices(images.shape, filter_shape, (pad_h, pad_w), stride)
    
    print("K:  ", np.shape(k))
    print("i   ", np.shape(i))
    print("j   ", np.shape(j))
    #print("K   ", k)
    #print("i  ", i)
    #print("j  ", j)
    
    # Get content from image at those indices
    cols = images_padded[:, k, i, j]
    channels = images.shape[1]
    print("cols_before_reshape:  ", np.shape(cols))
    # Reshape content into column shape
    
    cols = cols.transpose(1, 2, 0).reshape(filter_height * filter_width * channels, -1)
    return cols

# Reference: CS231n Stanford
def get_im2col_indices(images_shape, filter_shape, padding, stride=1):
    # First figure out what the size of the output should be
    batch_size, channels, height, width = images_shape
    filter_height, filter_width = filter_shape
    pad_h, pad_w = padding
    out_height = int((height + np.sum(pad_h) - filter_height) / stride + 1)
    out_width = int((width + np.sum(pad_w) - filter_width) / stride + 1)

    i0 = np.repeat(np.arange(filter_height), filter_width)
    i0 = np.tile(i0, channels)
    i1 = stride * np.repeat(np.arange(out_height), out_width)
    j0 = np.tile(np.arange(filter_width), filter_height * channels)
    j1 = stride * np.tile(np.arange(out_width), out_height)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(channels), filter_height * filter_width).reshape(-1, 1)

    return (k, i, j)

# Method which calculates the padding based on the specified output shape and the
# shape of the filters
def determine_padding(filter_shape, output_shape="same"):
    # No padding
    if output_shape == "valid":
        return (0, 0), (0, 0)
    # Pad so that the output shape is the same as input shape (given that stride=1)
    elif output_shape == "same":
        filter_height, filter_width = filter_shape

        # Derived from:
        # output_height = (height + pad_h - filter_height) / stride + 1
        # In this case output_height = height and stride = 1. This gives the
        # expression for the padding below.
        pad_h1 = int(math.floor((filter_height - 1)/2))
        pad_h2 = int(math.ceil((filter_height - 1)/2))
        pad_w1 = int(math.floor((filter_width - 1)/2))
        pad_w2 = int(math.ceil((filter_width - 1)/2))

        return (pad_h1, pad_h2), (pad_w1, pad_w2)

### Implement Backward Pass
Now you can start building your own backward function.

In [7]:
def backward_pass_conv(layer, accum_grad):
    '''
    TODO: Implement the backward_pass_fc here.
    
    Parameter:
    ----------
        accum_grad: gradient propogated back from the next layer
        
    Return:
    ----------
        accum_grad_result: gradient propogated back from the this layer
    '''
    accum_grad_result = np.zeros(layer.layer_input.shape)
     
        
    # the gradient of input
    # from 1*5*5*5 to 1*3*5*5 
    # full convolution: 5*7*7*7
    #accum_grad_result = convolve(accum_grad, layer.W, mode = 'full')
    
    # from 1*5*5*5 --- W: 5*3*3*3
    # from 1*5*5*5 --- S: 3*5*3*3
    #output.transpose(3,0,1,2))
    back_pass_w = layer.W.transpose(1,0,2,3)
    
    
    #test flip
    back_pass_w = np.flip(back_pass_w, (2,3))
    
    
    print("the shape of back_pass_w  ",np.shape(back_pass_w) )
    # the shape of test_w: 3*5*3*3 
   
    
    #initialize: 
    # batch * channels * height * weight = 1 * 3 * 5 * 5
    #grad_input_shape = layer.layer_input.shape
    
    # filter_shape: layer.filter_shape
    
    # the num of filters: 3
    n_filters = back_pass_w.shape[0]
    
    # padding: "same"
    padding = "same"
    # stride = 1
    stride = layer.stride
    
    
    #modify the forward_pass, consider the accum_grad as the input
    batch_size, channels, height, width = accum_grad.shape
    
       
    X_col_accum_grad = image_to_column(accum_grad, layer.filter_shape, stride=1, output_shape="same")
    
    W_col_back_pass = back_pass_w.reshape((n_filters, -1))
        
    
    result = W_col_back_pass.dot(X_col_accum_grad)
    # determine the output_shape 
    pad_h, pad_w = determine_padding(layer.filter_shape, output_shape= padding)
    output_height = (height + np.sum(pad_h) - layer.filter_shape[0]) / stride + 1
    output_width = (width + np.sum(pad_w) - layer.filter_shape[1]) / stride + 1
    
    # Reshape into (n_filters, out_height, out_width, batch_size)
    result_shape = (n_filters, int(output_height), int(output_width))
    result = result.reshape(result_shape + (batch_size, ))
    
    # Redistribute axises so that batch size comes first
    #print("the shape of output_forward_final:  ", np.shape(output.transpose(3,0,1,2)))
    accum_grad_result = result.transpose(3,0,1,2)
    print("the shape of accum_grad_result    ", np.shape(accum_grad_result))
    

    return accum_grad_result




### Test your implementation:
We use preloaded input, output, weight and bias tensor to test the implementation of your forward pas and backward pass. 

In [8]:
def conv_test():
    Conv2D.backward_pass = backward_pass_conv
    
    # np.load return the k,v pair of the name and value of numpy matrix
    data = np.load('test.npz')
    
    #print("the type of data  ",data)
    
    # read the input from npz file
    input_tensor = data['input_tensor']
    print("the type of input_tensor  ",np.shape(input_tensor))
    
    
    # read the forward pass result from npz file
    output_tensor = data['output_tensor']
    print("the type of output_tensor  ",np.shape(output_tensor))
    
    # read the target from npz file
    target_tensor = data['target_tensor']
    print("the type of target_tensor  ",np.shape(target_tensor))
    
    
    # read the backward pass result from npz file
    accum_grad = data['accum_grad']
    print("the type of accum_grad  ",np.shape(input_tensor))
    
    # read the preloaded weight and bias from npz file
    w0 = data['w0']
    print("the type of w0  ", np.shape(w0))
    # 
    W = data['W']
    print("the type of W  ", np.shape(W))
    

    # read the configuration from npz file
    filter_size = data['filter_size']
    filter_num = data['filter_num']
    print("the type of filter_size  ", filter_size)
    print("the type of filter_num  ", filter_num)
    
    
    # configure the 
    layer = Conv2D(n_filters=filter_num, filter_shape=(filter_size, filter_size), input_shape=input_tensor.shape)
    layer.W, layer.w0 = W, w0
    predict_tensor = layer.forward_pass(input_tensor)

    print("the type of predicted tensor  ", np.shape(predict_tensor))
    
    print("SE:  ",  np.shape( SE(predict_tensor, output_tensor))  )
    
    
    # Test the forward pass implementation
    if SE(predict_tensor, output_tensor).all() < 1e-1:
        print("[INFO] Testing Forward: Pass!")
    else:
        print("[WARN] Testing Forward: Fail!")
    
    # use the tensors read from the npz file to compute the loss
    loss = target_tensor - output_tensor
    
    print("the shape of loss  ", np.shape(loss))
    
    predict_accum_grad = layer.backward_pass(loss)
    
    print("the shape of SE   ", SE(predict_accum_grad, accum_grad) )
    
    
    
    # Test the backward pass implementation
    if SE(predict_accum_grad, accum_grad).all() < 1e-1:
        print("[INFO] Testing Backward: Pass!")
    else:
        print("[WARN] Testing Backward: Fail!")
    
    
    

In [9]:
conv_test()

the type of input_tensor   (1, 3, 5, 5)
the type of output_tensor   (1, 5, 5, 5)
the type of target_tensor   (1, 5, 5, 5)
the type of accum_grad   (1, 3, 5, 5)
the type of w0   (5, 1)
the type of W   (5, 3, 3, 3)
the type of filter_size   3
the type of filter_num   5
the pad_h:   (1, 1)
teh pad_w    (1, 1)
the shape of images_padded   (1, 3, 7, 7)
K:   (27, 1)
i    (27, 25)
j    (27, 25)
cols_before_reshape:   (1, 27, 25)
the shape of self.X_col   (27, 25)
the shape of self.W.col    (5, 27)
the shape of output_forward1:   (5, 25)
the shape of output_forward2:   (5, 5, 5, 1)
the shape of output_forward_final:   (1, 5, 5, 5)
the type of predicted tensor   (1, 5, 5, 5)
SE:   (1, 5, 5, 5)
[INFO] Testing Forward: Pass!
the shape of loss   (1, 5, 5, 5)
the shape of back_pass_w   (3, 5, 3, 3)
the pad_h:   (1, 1)
teh pad_w    (1, 1)
the shape of images_padded   (1, 5, 7, 7)
K:   (45, 1)
i    (45, 25)
j    (45, 25)
cols_before_reshape:   (1, 45, 25)
the shape of accum_grad_result     (1, 3, 5, 